In [ ]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datetime import datetime
import os

In [ ]:
# ✅ Detect MPS
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
project_root = os.path.abspath("..")

In [ ]:
# Load dataset
dataset_version = {
    "joined": os.path.join(project_root, "datasets/processed_joined_toxicity_data.csv"),
    "split": os.path.join(project_root, "datasets/processed_split_toxicity_data.csv"),
    "message_only": os.path.join(project_root, "datasets/processed_split_toxicity_data.csv")
}

# Three tokenizers
def tokenize_joined(example):
    return tokenizer(example["input_text"], padding="max_length", truncation=True, max_length=512)

def tokenize_split(example):
    return tokenizer(example["context"], example["message"], padding="max_length", truncation=True, max_length=512)

def tokenize_message_only(example):
    return tokenizer(example["message"], padding="max_length", truncation=True, max_length=512)

tokenize_funcs = {
    "joined": tokenize_joined,
    "split": tokenize_split,
    "message_only": tokenize_message_only
}

In [ ]:
results = {}
best_f1 = -1
best_model = None
best_version = ""

# Load tokenizer
tokenizer = RobertaTokenizer.from_pretrained("s-nlp/roberta_toxicity_classifier")

for version, tokenizer_func in tokenize_funcs.items():
    print(f"\nTraining version: {version}")
    df = pd.read_csv(dataset_version[version])  # Must contain 'input_text', 'context', 'message', 'label'
    dataset = Dataset.from_pandas(df)
    tokenized = dataset.map(tokenizer_func, batched=True)
    tokenized = tokenized.train_test_split(test_size=0.2)
    tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    model = RobertaForSequenceClassification.from_pretrained("s-nlp/roberta_toxicity_classifier", num_labels=2)
    model.to(device)

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        acc = accuracy_score(labels, preds)
        precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary', zero_division=0)
        return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

    training_args = TrainingArguments(
        output_dir="./results",
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=3e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=5,
        weight_decay=0.01,
        warmup_ratio=0.1,
        logging_dir="./logs",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["test"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    eval_result = trainer.evaluate()
    results[version] = eval_result

    if eval_result["eval_f1"] > best_f1:
        best_f1 = eval_result["eval_f1"]
        best_model = model
        best_version = version
        trainer.save_model(f"best_roberta_model_{version}")

# Print summary
print("\n--- Summary of Results ---")
for version, result in results.items():
    print(f"{version.upper()}: F1 = {result['eval_f1']:.4f}")

print(f"\nBest model: {best_version.upper()} (F1 = {best_f1:.4f})")